## Promotion Periods and Events

In [61]:
import pandas as pd

# Reading dataset
df = pd.read_csv("hotel.csv")

# Converting 'arrival_date' to datetime format
df['arrival_date'] = pd.to_datetime(df['arrival_date'], format='%Y-%m-%d')

In [62]:
unique_arrival_dates = df[['arrival_date', 'total_rooms_occupied']].drop_duplicates(subset='arrival_date')

# Addiing 'is_peak' column based on the 90th percentile of 'total_rooms_occupied' for unique dates
threshold = unique_arrival_dates['total_rooms_occupied'].quantile(0.90)
unique_arrival_dates['is_peak'] = unique_arrival_dates['total_rooms_occupied'] >= threshold

# Merging peak information with original dataframe
df = df.merge(unique_arrival_dates[['arrival_date', 'is_peak']], on='arrival_date', how='left')


In [63]:
unique_arrival_dates.head()

,arrival_date,total_rooms_occupied,is_peak
0,2015-07-01,128.0,False
4,2015-07-02,128.0,False
10,2015-07-03,71.0,False
26,2015-07-04,68.0,False
30,2015-07-05,89.0,False


In [64]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date,check_out_date,total_rooms_occupied,is_peak
0,City Hotel,0,6,2015,7,27,1,0,2,1,...,Transient,0.0,0,0,Check-Out,2015-07-03,2015-07-01,2015-07-03,128.0,False
1,City Hotel,1,88,2015,7,27,1,0,4,2,...,Transient,76.5,0,1,Canceled,2015-07-01,2015-07-01,2015-07-05,128.0,False
2,City Hotel,1,65,2015,7,27,1,0,4,1,...,Transient,68.0,0,1,Canceled,2015-04-30,2015-07-01,2015-07-05,128.0,False
3,City Hotel,1,92,2015,7,27,1,2,4,2,...,Transient,76.5,0,2,Canceled,2015-06-23,2015-07-01,2015-07-07,128.0,False
4,City Hotel,1,100,2015,7,27,2,0,2,2,...,Transient,76.5,0,1,Canceled,2015-04-02,2015-07-02,2015-07-04,128.0,False


In [65]:
num_is_peak = unique_arrival_dates['is_peak'].sum()
num_is_peak

79

In [66]:
def identify_event_periods(df, min_event_days=3):
    # Calculating the difference between consecutive 'arrival_date' entries
    df['date_diff'] = df['arrival_date'].diff().fillna(pd.Timedelta(days=0))
    
    event_periods = []
    current_period = []
    
    for index, row in df.iterrows():
        if row['is_peak']:
            if not current_period:
                current_period = [row['arrival_date']]
            elif row['date_diff'].days == 1:
                current_period.append(row['arrival_date'])
            else:
                if len(current_period) >= min_event_days:
                    event_periods.append((current_period[0], current_period[-1]))
                current_period = [row['arrival_date']]
        else:
            if len(current_period) >= min_event_days:
                event_periods.append((current_period[0], current_period[-1]))
            current_period = []
    
    # Adding the last event period if it meets the length criteria
    if len(current_period) >= min_event_days:
        event_periods.append((current_period[0], current_period[-1]))
    
    return event_periods

# Identifying event periods from the data
event_periods = identify_event_periods(df)


In [67]:
df['is_event'] = False
for start_date, end_date in event_periods:
    df.loc[(df['arrival_date'] >= start_date) & (df['arrival_date'] <= end_date), 'is_event'] = True

In [68]:
# Creating a new 'booking_date' column using lead time and arrival date
df['booking_date'] = df['arrival_date'] - pd.to_timedelta(df['lead_time'], unit='D')

booking_count = df.groupby('booking_date').size().reset_index(name='booking_count')

# Determining the 90th percentile threshold for booking counts
booking_threshold = booking_count['booking_count'].quantile(0.90)

# Adding 'is_peak_booking' column based on the 90th percentile of booking counts
booking_count['is_peak_booking'] = booking_count['booking_count'] >= booking_threshold

# Merging peak booking information with the original DataFrame
df = df.merge(booking_count[['booking_date', 'is_peak_booking']], on='booking_date', how='left')
df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,reservation_status,reservation_status_date,arrival_date,check_out_date,total_rooms_occupied,is_peak,date_diff,is_event,booking_date,is_peak_booking
0,City Hotel,0,6,2015,7,27,1,0,2,1,...,Check-Out,2015-07-03,2015-07-01,2015-07-03,128.0,False,0 days,False,2015-06-25,False
1,City Hotel,1,88,2015,7,27,1,0,4,2,...,Canceled,2015-07-01,2015-07-01,2015-07-05,128.0,False,0 days,False,2015-04-04,False
2,City Hotel,1,65,2015,7,27,1,0,4,1,...,Canceled,2015-04-30,2015-07-01,2015-07-05,128.0,False,0 days,False,2015-04-27,False
3,City Hotel,1,92,2015,7,27,1,2,4,2,...,Canceled,2015-06-23,2015-07-01,2015-07-07,128.0,False,0 days,False,2015-03-31,False
4,City Hotel,1,100,2015,7,27,2,0,2,2,...,Canceled,2015-04-02,2015-07-02,2015-07-04,128.0,False,1 days,False,2015-03-24,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71176,City Hotel,0,23,2017,8,35,30,2,5,2,...,Check-Out,2017-09-06,2017-08-30,2017-09-06,304.0,False,0 days,False,2017-08-07,False
71177,City Hotel,0,102,2017,8,35,31,2,5,3,...,Check-Out,2017-09-07,2017-08-31,2017-09-07,304.0,False,1 days,False,2017-05-21,False
71178,City Hotel,0,34,2017,8,35,31,2,5,2,...,Check-Out,2017-09-07,2017-08-31,2017-09-07,304.0,False,0 days,False,2017-07-28,False
71179,City Hotel,0,109,2017,8,35,31,2,5,2,...,Check-Out,2017-09-07,2017-08-31,2017-09-07,304.0,False,0 days,False,2017-05-14,False


In [69]:
import pandas as pd

def find_promotion_periods(df, event_periods, min_promotion_days=3):
    promotion_periods = []

    for start_date, end_date in event_periods:
        # Filtering dataFrame based on event period
        period_df = df[(df['arrival_date'] >= start_date) & (df['arrival_date'] <= end_date)]
        
        # Sorting by booking_date to ensure proper sequence
        period_df = period_df.sort_values(by='booking_date')
        
        # Creating a column to mark consecutive peak booking periods
        period_df['consecutive_peak'] = (period_df['is_peak_booking'] &
                                         (period_df['booking_date'].diff().fillna(pd.Timedelta(days=0)).dt.days == 1))
        
        # Finding continuous sequences
        current_period = []
        for index, row in period_df.iterrows():
            if row['is_peak_booking']:
                if not current_period:
                    current_period = [row['booking_date']]
                elif row['consecutive_peak']:
                    current_period.append(row['booking_date'])
                else:
                    # Check if the current period is long enough
                    if len(current_period) >= min_promotion_days:
                        promotion_periods.append((current_period[0], current_period[-1], start_date, end_date))
                    current_period = [row['booking_date']]
            else:
                if len(current_period) >= min_promotion_days:
                    promotion_periods.append((current_period[0], current_period[-1], start_date, end_date))
                current_period = []
        
        # Final check for the last period
        if len(current_period) >= min_promotion_days:
            promotion_periods.append((current_period[0], current_period[-1], start_date, end_date))
    
    return promotion_periods

In [70]:
df['is_promotion_period'] = False
for start_date, end_date in event_periods:
    df.loc[(df['arrival_date'] >= start_date) & (df['arrival_date'] <= end_date), 'is_promotion_period'] = True

In [71]:
def print_promotion_periods_with_events(promotion_periods, event_periods):
    print("Promotion Periods and Corresponding Event Dates:")
    for promo_start, promo_end, event_start, event_end in promotion_periods:
        print(f"Promotion Period: {promo_start.date()} to {promo_end.date()}")
        print(f"Event Period: {event_start.date()} to {event_end.date()}")
        print("-" * 40)
promotion_periods = find_promotion_periods(df, event_periods)
print_promotion_periods_with_events(promotion_periods, event_periods)

Promotion Periods and Corresponding Event Dates:
Promotion Period: 2016-02-09 to 2016-02-11
Event Period: 2016-04-28 to 2016-04-30
----------------------------------------
Promotion Period: 2016-02-09 to 2016-02-11
Event Period: 2016-04-28 to 2016-04-30
----------------------------------------
Promotion Period: 2016-02-09 to 2016-02-11
Event Period: 2016-04-28 to 2016-04-30
----------------------------------------
Promotion Period: 2017-01-09 to 2017-01-12
Event Period: 2017-04-15 to 2017-04-17
----------------------------------------
Promotion Period: 2017-01-09 to 2017-01-11
Event Period: 2017-04-28 to 2017-04-30
----------------------------------------
Promotion Period: 2017-01-18 to 2017-01-20
Event Period: 2017-04-28 to 2017-04-30
----------------------------------------
Promotion Period: 2017-01-09 to 2017-01-11
Event Period: 2017-05-04 to 2017-05-06
----------------------------------------
Promotion Period: 2017-01-09 to 2017-01-11
Event Period: 2017-05-04 to 2017-05-06
--------